### Import

In [69]:
import pandas as pd
import requests
import folium
from branca.element import Figure
from lib import get_popup_text

### Import data from the URL

In [70]:
url = "https://data.grandlyon.com/fr/datapusher/ws/grandlyon/eco_economie.ecomangerlocal_latest/all.csv?maxfeatures=-1"
response = requests.get(url)
if response.status_code == 200:
	df = pd.read_csv(url, sep=';', decimal=',')
	print('import successfull')
else:
	print(f"Failed to retrieve data from url {url}. Status code: {response.status_code}")

import successfull


In [71]:
df.dropna(inplace=True, axis=1, how='all')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   identifiant       137 non-null    int64  
 1   nom               137 non-null    object 
 2   adresse           137 non-null    object 
 3   insee             137 non-null    int64  
 4   commune           137 non-null    object 
 5   type              137 non-null    object 
 6   joursouverture    126 non-null    object 
 7   horaires          126 non-null    object 
 8   produits          137 non-null    object 
 9   origine_produits  137 non-null    object 
 10  label             94 non-null     object 
 11  datemaj           137 non-null    object 
 12  gid               137 non-null    int64  
 13  lon               137 non-null    float64
 14  lat               137 non-null    float64
dtypes: float64(2), int64(3), object(10)
memory usage: 16.2+ KB


In [72]:
df.head(10)

,identifiant,nom,adresse,insee,commune,type,joursouverture,horaires,produits,origine_produits,label,datemaj,gid,lon,lat
0,24,Gaec des Sables Rouges,12 route de La Tour de Salvagny,69072,Dardilly,Vente à la ferme,NaN,NaN,"['Légumes', 'Fruits']",Département 69,NaN,2022-03-28,1185,4.750042,45.822355
1,36,Nadine Jacquet,14 rue des verchères,69163,Quincieux,Vente à la ferme,NaN,NaN,['Fruits'],Département 69,NaN,2022-03-28,1194,4.759349,45.909030
2,50,EARL Bossy,3 route de Toussieu,69290,Saint-Priest,Vente à la ferme,NaN,NaN,"['Légumes', 'Oeufs', 'Miel']",Département 69,NaN,2022-03-28,1202,4.980986,45.700137
3,57,La Ferme Perraud,32 rue des Docteurs Cordier,69389,Lyon 9e Arrondissement,Vente à la ferme,NaN,NaN,"['Légumes', 'Fruits']",Département 69,NaN,2022-03-28,1128,4.830228,45.801796
4,310,Episol du CADIS,29 Avenue Jean Jaurès,69275,Décines-Charpieu,Epicerie sociale et solidaire,"['Lu', 'Ma', 'Me', 'Je', 'Ve']","['Mo,Tu,We,Th,Fr 00:00-00:00']",['Epicerie'],France,[],2022-04-27,1211,4.939377,45.768379
5,555,EARL Gontel,Place du Chanoine Boursier,69266,Villeurbanne,Producteur du marché (Cultive ses produits et ...,NaN,NaN,['Légumes'],Département du Rhône,['AB - agriculture biologique'],2022-05-04,1212,4.878384,45.768834
6,559,Gaec Les Biquettes des Orlets,Place du Chanoine Boursier,69266,Villeurbanne,Producteur du marché (Cultive ses produits et ...,['Ma'],['Tu 8:00-12:00'],"['Fromage et produits laitiers', 'Oeufs']",Département du Rhône,['Autres labels'],2022-05-05,1215,4.878384,45.768834
7,562,Claron Père et Fils,Place du Chanoine Boursier,69266,Villeurbanne,Producteur du marché (Cultive ses produits et ...,"['Ma', 'Je', 'Sa']","['Tu,Th,Sa 8:00-12:00']","['Fruits', 'Légumes', 'Boissons']",Département du Rhône,NaN,2022-05-05,1216,4.878384,45.768834
8,565,Ferme de Menajoc,2 Place Jean Jaurès,69152,Pierre-Bénite,Producteur du marché (Cultive ses produits et ...,['Ve'],['Fr 16:00-19:00'],"['Boulangerie', 'Autre']",Département du Rhône,['AB - agriculture biologique'],2022-05-05,1218,4.823765,45.703753
9,566,La Ferme de Piamot,1451 Rue de la Grande Verchère,69278,Genay,Vente à la ferme,"['Lu', 'Ma', 'Me', 'Je', 'Ve', 'Sa', 'Di']","['Mo,Tu,We,Th,Fr,Sa,Su 01:00-00:59']","['Viande', 'Oeufs', 'Autre']",Département du Rhône,['AB - agriculture biologique'],2022-05-05,1219,4.858874,45.899410


In [73]:
df['type'].unique()

array(['Vente à la ferme', 'Epicerie sociale et solidaire',
       'Producteur du marché (Cultive ses produits et les vend)', 'AMAP',
       'Magasin de producteurs',
       'Revendeur du marché (Achète des produits et les revend)'],
      dtype=object)

In [74]:
df['type'] = df['type'].str.replace(r'\s*\(.*\)', '', regex=True)
df['type'].unique()

array(['Vente à la ferme', 'Epicerie sociale et solidaire',
       'Producteur du marché', 'AMAP', 'Magasin de producteurs',
       'Revendeur du marché'], dtype=object)

In [75]:
icon_colors = {
    'Vente à la ferme': ['green', 'fa-cheese'],
    'Epicerie sociale et solidaire': ['blue', 'fa-carrot'],
    'Producteur du marché': ['red', 'fa-apple-whole'],
    'AMAP': ['purple', 'fa-egg'],
    'Magasin de producteurs': ['orange', 'fa-seedling'],
    'Revendeur du marché': ['pink', 'fa-apple-whole']
}

In [76]:
type_layer = df['type'].unique()

In [77]:
lyon = [45.7640, 4.8357]
map = folium.Map(location=lyon, zoom_start=12)
df.apply(lambda row: folium.Marker(
        location=[row['lat'], row['lon']],
        popup=get_popup_text(row),
        icon=folium.Icon(
            icon=icon_colors[row['type']][1],
            color=icon_colors[row['type']][0],
            prefix='fa')
        ).add_to(map), axis=1)
map.save('map.html')

In [78]:
# fig = Figure(width=1200, height=800)
# fig.add_child(map)
map